In [9]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os
import time

In [10]:
post_ids_df = pd.read_csv('data/vebay69/clean/post_ids.csv')
post_ids_df.head()

,post_id
0,685638947004646
1,685577330344141
2,685111467057394
3,684686633766544
4,684603707108170


In [11]:
def extract_reactor(reactors, post_id):
    cnt_reactor = 0
    global reactor_list
    for reactor in reactors:
        reactor['post_id'] = post_id
        reactor_list.append(reactor)
        cnt_reactor += 1
        # time.sleep(np.random.uniform(0.5, 2.5))
    return cnt_reactor

In [12]:
index_post_id_file_path = os.path.join('data','vebay69','url','i_post_id_react.txt')

### Rerun with change cookie

In [13]:
fs.set_cookies('./cookies/cookie_1.txt')
# fs.enable_logging()

### Rerun 

In [14]:
NUMBER_POST = 5
post_list = []
reactor_list = []
total_reactor_real = 0

In [15]:
index_post_id = utils.read_index_file(file_path=index_post_id_file_path)
print(index_post_id)
posts_ids = utils.get_post_id_list(df=post_ids_df, start=index_post_id, length=NUMBER_POST)
posts_ids

4


[684603707108170,
 684191557149385,
 683955240506350,
 682996903935517,
 683571943878013]

In [16]:
try:
    for post in fs.get_posts(
        post_urls=posts_ids,
        options={
            "allow_extra_requests": True,
            "reactors": True,
            "reactors": "generator",
            "reactions": True
        },
        timeout=120,
    ):
        print(post)
        num_sharer = extract_reactor(post['reactors'], post['original_request_url'])
        post_list.append(post)
        print(f'Extracted {num_sharer}/{post["reaction_count"]} reactors')
        total_reactor_real += post['reaction_count']
        utils.sleep(np.random.randint(60, 180))
        print()
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

/workspaces/analyze-facebook-page/lib/facebook_scraper/facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")


{'original_request_url': 684603707108170, 'post_url': 'https://facebook.com/story.php?story_fbid=684603707108170&id=100066756416622', 'post_id': 684603707108170, 'text': 'Team vô sản ✌️\n#Muonggg', 'post_text': 'Team vô sản ✌️\n#Muonggg', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2023, 11, 18, 8, 34, 17), 'timestamp': 1700296457, 'image': 'https://scontent-sin6-1.xx.fbcdn.net/v/t39.30808-6/401741542_684603540441520_8889609901616083207_n.jpg?stp=cp0_dst-jpg_e15_fr_q65&_nc_cat=111&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=B29g7qwhlUYAX_Clu9L&_nc_ht=scontent-sin6-1.xx&oh=00_AfCcKa5pHnMgOlPcvoP4EqylPEUduDwavqLGaTfB6cfaDw&oe=656AD96E&manual_redirect=1', 'image_lowquality': 'https://scontent-sin6-1.xx.fbcdn.net/v/t39.30808-6/401741542_684603540441520_8889609901616083207_n.jpg?stp=cp0_dst-jpg_e15_p320x320_q65&_nc_cat=111&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=B29g7qwhlUYAX_Clu9L&_nc_ht=scontent-sin6-1.xx&oh=00_AfDjZ-_UgDHdCqPb32Yw52_G6BRTq7B63cDVISbBlG

/workspaces/analyze-facebook-page/lib/facebook_scraper/facebook_scraper.py:908: UserWarning: Facebook served mbasic/noscript content unexpectedly on https://m.facebook.com/ufi/reaction/profile/browser/fetch/?ft_ent_identifier=682996903935517&limit=50&shown_ids=100076259955137%2C100076277623610%2C100076295562564%2C100076294753513%2C100078514170269%2C100078509457294%2C100076312151748%2C100078827265473%2C100078824589544%2C100078820838961%2C100078820216595%2C100078625131769%2C100078600415414%2C100078976404100%2C100078975234329%2C100078973222765%2C100078967703723%2C100078942580759%2C100078835150432%2C100078833839820%2C100078831645838%2C100079002447705%2C100078998687777%2C100078997519738%2C100078997460171%2C100078996715087%2C100078995719834%2C100078989844740%2C100078984979373%2C100078984916915%2C100078983504816%2C100078982458793%2C100078982377296%2C100078981604599%2C100079047011834%2C100079046602919%2C100079041902791%2C100079035777267%2C100079029397543%2C100079028840454%2C100079028317634%2C1

Extracted 333/1130 reactors
	Sleep 102 seconds

{'original_request_url': 683571943878013, 'post_url': 'https://facebook.com/story.php?story_fbid=683571943878013&id=100066756416622', 'post_id': 683571943878013, 'text': 'Quên hoài…\n#Muonggg\n\nAlways forget...\n#Muonggg', 'post_text': 'Quên hoài…\n#Muonggg\n\nAlways forget...\n#Muonggg', 'shared_text': '', 'original_text': 'Quên hoài…\n#Muonggg', 'time': datetime.datetime(2023, 11, 16, 8, 37, 17), 'timestamp': 1700123837, 'image': 'https://scontent-sin6-1.xx.fbcdn.net/v/t39.30808-6/401634735_683571700544704_6049515324962041659_n.jpg?stp=cp0_dst-jpg_e15_fr_q65&_nc_cat=107&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=UtMPKo1D81MAX_hizqx&tn=E0gD5szyVABlKnmA&_nc_ht=scontent-sin6-1.xx&oh=00_AfCWReC47IJ5n9Dhq-_qh6louBNmSK7XuP6BoT25zV6mvQ&oe=656B3332&manual_redirect=1', 'image_lowquality': 'https://scontent-sin6-1.xx.fbcdn.net/v/t39.30808-6/401634735_683571700544704_6049515324962041659_n.jpg?stp=cp0_dst-jpg_e15_p320x320_q65&_nc_cat=107&ccb=

/workspaces/analyze-facebook-page/lib/facebook_scraper/facebook_scraper.py:908: UserWarning: Facebook served mbasic/noscript content unexpectedly on https://m.facebook.com/ufi/reaction/profile/browser/fetch/?ft_ent_identifier=683571943878013&limit=50&shown_ids=100075904776613%2C100075907986469%2C100075912666232%2C100075922805730%2C100075919085805%2C100075947733028%2C100075947164555%2C100075936035048%2C100075922895883%2C100075980192922%2C100075968793422%2C100075967923580%2C100075961773818%2C100075957724072%2C100075951544221%2C100076255875411%2C100076248765555%2C100076227677002%2C100076079812975%2C100076032690744%2C100075995701953%2C100078600415414%2C100078514170269%2C100078509457294%2C100076312151748%2C100076295562564%2C100076294753513%2C100076277623610%2C100076259955137%2C100078975234329%2C100078973222765%2C100078967703723%2C100078942580759%2C100078835150432%2C100078833839820%2C100078831645838%2C100078827265473%2C100078824589544%2C100078820838961%2C100078820216595%2C100078763119919%2C1

Extracted 353/1498 reactors
	Sleep 69 seconds



In [18]:
print(NUMBER_POST)
print(f"Số posts đã crawl: {len(post_list)}")
print(f"Số sharers đã crawl: {len(reactor_list)}/{total_reactor_real}")

5
Số posts đã crawl: 5
Số sharers đã crawl: 2730/8990


In [19]:
if reactor_list:
    reactor_df = utils.save_data(
        data_list=reactor_list,
        folder_path=os.path.join('data','vebay69','raw'),
        type="reactors"
    )
utils.write_index_file(
    file_path=index_post_id_file_path,
    index=index_post_id + len(post_list)
)

Save reactors data: ./data/vebay69/raw/reactors_2023-11-28_14-42-33.csv
Save index 9 to resume: ./data/vebay69/url/i_post_id_react.txt
